## coklu dogrusal regresyon - tahmin


Model denklemi:

Sales = 2.97 + TV*0.04 + radio*0.18 + newspaper\*0.002

Örneğin 30 birim TV harcaması, 10 birim radio harcaması, 40 birim de gazete harcaması olduğunda satışların tahmini değeri ne olur?


In [1]:
import pandas as pd
import warnings

warnings.simplefilter(action="ignore", category=UserWarning)

ad = pd.read_csv("Advertising.csv", usecols=[1, 2, 3, 4])
df = ad.copy()
df.head()

# coklu dogrusal regresyonda test-train diye ayirmalara baslayacagiz

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

# test-train icin gerekli importlar

In [3]:
x = df.drop("sales", axis=1)
x[0:10]

# sales bagimli degiskenini silerek sadece bagimsiz degiskenleri sectik

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4
5,8.7,48.9,75.0
6,57.5,32.8,23.5
7,120.2,19.6,11.6
8,8.6,2.1,1.0
9,199.8,2.6,21.2


In [4]:
y = df["sales"]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.20, random_state=42
)

# esittirin sag tarafinda yapilan butun islemlerin ciktisini, sol taraftaki degiskenlere kaydeder
# x = bagimsiz degisken, y = bagimli degisken
# test_size = 80'e 20 olarak ayiracagimiz icin 0.20 degerini girdik
# her uretmede farkli degerler cikmamasi icin random_state = 42 gibi bir deger girdik

In [6]:
x_train.shape

(160, 3)

In [7]:
y_train.shape

(160,)

In [8]:
x_test.shape

(40, 3)

In [9]:
y_test.shape

(40,)

In [10]:
training = df.copy()
training.shape

# veri setimizin ilk hali

(200, 4)

#### ========================================================

#### statsmodels ile model kurma

- yorumlama gibi bir ihtiyacimiz varsa statsmodels daha iyi

- diger fonksiyonlarla da uyumlu, diger makine ogrenmesi algoritmalariyla da uyumlu bir islem yapilmak istenirse scikit learn


In [11]:
import statsmodels.api as sm

lm = sm.OLS(y_train, x_train)
model = lm.fit()
model.summary()

# r-squared ve adj. r-squared degerlerine bakildiginde, satis degiskeninin degiskenliginin %98'ini aciklayabiliyoruz anlamina geliyor.
# f-statistic ve prob degerine baktigimizda modelin anlamli oldugunu goruyoruz.

# coef degerlerine baktigimizda, tv icin yapilacak 1 birimlik harcama artisi satislarda ortalama 0.0531 artisa sebep olacaktir.

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2935.
Date:                Mon, 26 Jun 2023   Prob (F-statistic):                   1.28e-137
Time:                        02:29:59   Log-Likelihood:                         -336.65
No. Observations:                 160   AIC:                                      679.3
Df Residuals:                     157   BIC:                                      688.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
model.summary().tables[1]

# tables[1] kullanimi kat sayilari verir

,coef,std err,t,P>|t|,[0.025,0.975]
TV,0.0531,0.001,36.467,0.000,0.050,0.056
radio,0.2188,0.011,20.138,0.000,0.197,0.240
newspaper,0.0239,0.008,3.011,0.003,0.008,0.040


#### ========================================================

#### varyans aciklama basarisi ve tahmin basarisi


varyans aciklama basarisi = yukaridaki tablodaki r-squared ve adj. r-squared degerleri


modellerin tahmin basarisini degerlendirmek icin mse, rmse gibi degerlere bakilir


#### ========================================================

#### scikit-learn model


In [13]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
model = lm.fit(x_train, y_train)

# lm.fit() icindeki ilk deger bagimsiz degiskenleri ifade eder

In [14]:
model.intercept_

# sabit kat sayiya erisim

2.9790673381226256

In [15]:
model.coef_

# butun bagimsiz degiskenlerin kat sayilarina erisim

array([0.04472952, 0.18919505, 0.00276111])

# ================================

# TAHMIN KISMI BURADAN BASLIYOR


In [16]:
yeni_veri = [[30], [10], [40]]
yeni_veri = pd.DataFrame(yeni_veri).T

In [17]:
model.predict(yeni_veri)

# TVde 30, radyoda 10, gazetede 40 birim harcama yaptigimizda satislarimizin beklenen degeri 6.32... olacaktir.

array([6.32334798])

In [18]:
import numpy as np
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_train, model.predict(x_train)))
rmse

# modelimizin tahmin basarisi
# bu egitim hatasidir

# y_train = gercek y degerleri
# model.predict(x_train) = bagimsiz degiskenlere bakarak modelin tahmin ettigi y degerleri

1.644727765644337

In [19]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(x_test)))
rmse

# bu test hatasidir

1.7815996615334504